In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
.libPaths(c("/content/drive/MyDrive/Colab Notebooks/R_lib", .libPaths()))

In [1]:
.libPaths(c("/content/drive/MyDrive/Colab Notebooks/R_lib", .libPaths()))
library(limma)
library(dplyr)
library(tidyverse)
library(pheatmap)
library(EnhancedVolcano)
library(ggplot2)
library(plotly)
library(EnhancedVolcano)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.5.1     ✔ stringr   1.5.1
✔ lubridate 1.9.4     ✔ tibble    3.2.1
✔ purrr     1.0.4     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: ggrepel


Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout




In [2]:
filtered_num_mat = read.csv('filtered_num_mat.csv', row.names = 1)
exp_log = log2(filtered_num_mat + 1)

exp_scaled_t = scale(t(exp_log))
exp_scaled = t(exp_scaled_t)

In [3]:
metadata = read.csv('metadata_GSE60427.csv')

In [4]:
unique(metadata$condition)

selected_metadata2 = metadata[metadata$condition %in% c("H.P_Infection", "Control"), ]
dim(selected_metadata2)

selected_columns2 = colnames(exp_scaled)[colnames(exp_scaled) %in% selected_metadata2$ID]

filtered_data_scaled2 = exp_scaled[, selected_columns2]
dim(filtered_data_scaled2)

[1] "Control"       "H.P_Infection" "H.P_Gastritis" "IM"

[1] 16  4

[1] 28872    16

In [5]:
unique(selected_metadata2$condition)

design = model.matrix(~ 0 + condition, data = selected_metadata2)

colnames(design) <- gsub("condition", "", colnames(design))

unique(selected_metadata2$condition)

# Fit the model
fit = lmFit(filtered_data_scaled2, design)

# Create contrasts for differential expression
contrast_matrix = makeContrasts(H.PyloriInfection_Vs_Control = H.P_Infection - Control, levels = design)

# Apply contrasts
fit2 = contrasts.fit(fit, contrast_matrix)

# Perform empirical Bayes moderation
fit2 = eBayes(fit2)

[1] "Control"       "H.P_Infection"

[1] "Control"       "H.P_Infection"

In [6]:
results_H.PyloriInfection_Vs_Control = topTable(fit2, adjust.method = "fdr", number = Inf, sort.by = "p")

# View DEGs
results_H.PyloriInfection_Vs_Control

,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
C4orf7,2.011147,-0.4776078,9.216323,3.683223e-08,0.001063420,8.711157
CD180,1.904532,-0.5926451,8.578542,1.047834e-07,0.001512653,7.794675
CD19,1.911310,-0.5733196,8.023891,2.710238e-07,0.001877313,6.948548
LCN2,2.041719,-0.4898282,7.993378,2.858971e-07,0.001877313,6.900636
FCRL2,1.939792,-0.5893029,7.920251,3.251097e-07,0.001877313,6.785227
CR2,1.923676,-0.4793143,7.736375,4.505419e-07,0.002045413,6.491358
CXCL13,1.891559,-0.4763117,7.560671,6.179563e-07,0.002045413,6.205607
HLA-DRA,1.843104,-0.5222592,7.547494,6.328792e-07,0.002045413,6.183981
FAM159A,1.755974,-0.5265288,7.501420,6.880727e-07,0.002045413,6.108152
